In [ ]:
%load_ext autoreload
%autoreload 2

# Setup

In [ ]:
%matplotlib inline

# Imports.
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import networkx as nx
import pandas as pd
import pickle
from mpl_toolkits.mplot3d import Axes3D

# Important for multiprocessing.
import torch
torch.set_num_threads(1)

# General plotting things.
from plot import get_3d_subplot_axs
from plot import get_figsize, set_figsize
from plot import plot_df_trisurf
from plot import plot_lattice

default_w, default_h = get_figsize()

# Experiment imports.
from collections import OrderedDict
from gridsearch import experiment, load_experiment

# Dataset.
import dataset as ds

u_train, y_train = ds.NARMA(sample_len = 2000)
u_test, y_test = ds.NARMA(sample_len = 3000)
dataset = [u_train, y_train, u_test, y_test]
ds.dataset = dataset

# Distance functions.
from matrix import euclidean

euc = euclidean
def inv(x, y): return 1/euclidean(x, y)
def inv_squared(x, y): return 1/euclidean(x, y)**2
def inv_cubed(x, y): return 1/euclidean(x, y)**3

# Oftentimes for debugging purposes.
from ESN import ESN, Distribution
from metric import evaluate_esn

from notebook import default_plot_settings
default_plot_settings()

# Background

## The NARMA10 time series

In [ ]:
%%script false --no-raise-error

from notebook import plot_NARMA10
plot_NARMA10(range=[50, 200])

## NARMA nonlinearity

In [ ]:
from notebook import plot_NARMA_nonlinearity
plot_NARMA_nonlinearity()

## MG17 example

In [ ]:
from notebook import plot_mg17_example
plot_mg17_example()

## Benchmark example

In [ ]:
from notebook import plot_benchmark_example
plot_benchmark_example(save=False)

# Experiments: Random Geometric Graphs

## Example RGG

In [ ]:
from notebook import plot_rgg_example
plot_rgg_example(save=False)

In [ ]:
from notebook import plot_rgg_example_connected
plot_rgg_example_connected(save=False)

## Size of the Graph Volume

In [ ]:
%%script false --no-raise-error

from notebook import rgg_volume_size
rgg_volume_size()

In [ ]:
from notebook import plot_rgg_volume_size
plot_rgg_volume_size(std=False)

## Echo State Networks with nodes in metric space

Undirected geometric graphs with nodes randomly sampled uniformly in the  
underlying space [0, 1)^d.  

In [ ]:
%%script false --no-raise-error

from notebook import rgg_dist_performance
rgg_dist_performance()

In [ ]:
from notebook import plot_rgg_dist_performance
plot_rgg_dist_performance(agg='mean', file_name='RGG-dist-performance-mean.png', std=False)
plot_rgg_dist_performance(agg='min', file_name='RGG-dist-performance-min.png')

In [ ]:
%%script false --no-raise-error

from notebook import rgg_dist_mc
rgg_dist_mc()

In [ ]:
from notebook import plot_rgg_dist_mc
plot_rgg_dist_mc(std=False)

In [ ]:
%%script false --no-raise-error

from notebook import rgg_dist_performance_is
rgg_dist_performance_is()

In [ ]:
from notebook import plot_rgg_dist_performance_is
plot_rgg_dist_performance_is(std=False)

In [ ]:
%%script false --no-raise-error

from notebook import performance_restoration
performance_restoration()

In [ ]:
from notebook import plot_performance_restoration
plot_performance_restoration(std=False)

In [ ]:
from notebook import plot_performance_restoration_comparison
plot_performance_restoration_comparison(std=False)

In [ ]:
from notebook import plot_new_weight_distribution
plot_new_weight_distribution()

# Experiments: Regular Tilings

## Performance of standard regular tilings

In [ ]:
from notebook import plot_regular_tilings
plot_regular_tilings(save=False)

In [ ]:
%%script false --no-raise-error

from notebook import regular_tilings_performance
regular_tilings_performance()

In [ ]:
from notebook import plot_regular_tilings_performance
plot_regular_tilings_performance(std=False)

In [ ]:
%%script false --no-raise-error

from notebook import regular_tilings_performance_is
regular_tilings_performance_is()

In [ ]:
from notebook import plot_regular_tilings_performance_is
plot_regular_tilings_performance_is(std=False)

In [ ]:
esn = ESN(hidden_nodes=81, w_res_type='tetragonal')
evaluate_esn(ds.dataset, esn, plot=True, plot_range=[0, 100])

## Regular tilings with inhibitory connections

Hardly interesting.  

## Regular tilings with directed connections

What happens if we make a fraction of the edges of the lattice directed?  

In [ ]:
from notebook import plot_directed_lattice
plot_directed_lattice()

Performance-wise, we should, according to previous work, expect at least some  
improvement.  

In [ ]:
%%script false --no-raise-error

from notebook import directed_regular_tilings_performance
directed_regular_tilings_performance()

In [ ]:
from notebook import plot_directed_regular_tilings_performance
plot_directed_regular_tilings_performance(std=False)

It would seem that, the performance of the lattices match that of the standard  
echo state network for the NARMA 10 task. What about memory capacity and QGU?  

## Physical perspective: global input scheme

In [ ]:
from notebook import global_input_scheme_performance
global_input_scheme_performance()

In [ ]:
from notebook import plot_global_input_activations
plot_global_input_activations(save=False)

## Dynamics in Square Grids

In [ ]:
%%script false --no-raise-error

from notebook import kq_gen
kq_gen()

In [ ]:
from notebook import plot_square_grid_kq_gen
plot_square_grid_kq_gen(std=False)

In [ ]:
%%script false --no-raise-error

from notebook import run_mg_17
run_mg_17()

In [ ]:
from notebook import plot_mg_17
plot_mg_17(std=False)

## Performance comparison with ESN

In [ ]:
%%script false --no-raise-error

from notebook import directed_lattice_performance
directed_lattice_performance()

In [ ]:
from notebook import plot_directed_lattice_performance
plot_directed_lattice_performance(std=False)

In [ ]:
%%script false --no-raise-error

from notebook import unique_weights
unique_weights()

In [ ]:
from notebook import print_unique_weights
print_unique_weights()

## Reservoir robustness: removing nodes gradually

In [ ]:
%%script false --no-raise-error

from notebook import node_removal_impact
node_removal_impact()

In [ ]:
from notebook import plot_node_removal_impact
plot_node_removal_impact()

In [ ]:
from notebook import remove_nodes_performance
remove_nodes_performance()

In [ ]:
%%script false --no-raise-error

from notebook import general_esn_shrink
general_esn_shrink()

In [ ]:
from notebook import remove_esn_nodes_performance
remove_esn_nodes_performance()

Now, how do these lattices look?  

In [ ]:
from notebook import plot_node_removal
plot_node_removal(save=False)

How does the ESN look?  

In [ ]:
from notebook import plot_esn_node_removal
plot_esn_node_removal()

## Growing reservoirs: adding nodes incrementally

In [ ]:
from notebook import plot_growth
plot_growth(save=False)

In [ ]:
from notebook import plot_growth_performance
plot_growth_performance()

## Gradually making edges undirected

In [ ]:
%%script false --no-raise-error

from notebook import making_edges_undirected_performance
making_edges_undirected_performance()

In [ ]:
from notebook import plot_making_edges_undirected_performance
plot_making_edges_undirected_performance()

In [ ]:
from notebook import plot_making_edges_undirected
plot_making_edges_undirected()

## Example of great performance

In [ ]:
from notebook import plot_good_performance
plot_good_performance()